*Construct Question Answering System capable of answering questions related to corona virus specifically questions like What are symptoms of corona, What do we know about COVID-19 risk factors, How to prevent corona etc. i.e. some corona literature based questions.*

https://medium.com/@aakashgoel12/question-answering-system-on-corona-approach-01-6ef9799695cb

naive approach to solve the problem

Lets start on below idea for our 1st end to end solution:
Download all corona articles and simply split it into sentences.<>

Filter relevant sentences based on user’s query.
Feed each relevant sentence as context to BERT Model fine-tuned on SQUAD data-set and show selected answers based on score given by model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.Download data from Kaggle:

In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

#/content/kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"paridhigola","key":"d2c256f2334c666285afa5ce0ccc66df"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge -p /content/drive/MyDrive/Question_Answering_System/COVID-19

In [ ]:
#! unzip /content/drive/MyDrive/Question_Answering_System/COVID-19/CORD-19-research-challenge.zip -d /content/drive/MyDrive/Question_Answering_System/COVID-19

In [ ]:
## Storing file absolute path in list
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import json,os,gc


#path1 = 'biorxiv_medrxiv/biorxiv_medrxiv/'
#path2 = 'comm_use_subset/comm_use_subset/'
#path3 = 'custom_license/custom_license/'
#path4 = 'noncomm_use_subset/noncomm_use_subset/'


#paths = [path1,path2,path3,path4]
path = '/content/drive/MyDrive/Question_Answering_System/COVID-19/document_parses/pdf_json/'
file_names = []
#for path in paths:
  #temp_file_names = os.listdir(path)
  #file_names.extend([path+file_name for file_name in temp_file_names])


temp_file_names = os.listdir(path)
for file_name in temp_file_names:
  file_names.extend([path+file_name])
len(file_names)

12017

In [ ]:
#nltk.download('all')

### 2. Parse data
 Downloaded articles are in .json format, need to convert all data into rows and columns where each rows signify one article.

In [ ]:
## function to parse data
from time import time
from copy import deepcopy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import wordnet,stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import os,re,multiprocessing,joblib
from multiprocessing import Pool
from collections import defaultdict


stopwords = set(stopwords.words('english'))
# nltk.download('all')


def file_content(file_path):
  abstract='';body_text = '';error_count = 0
  if os.path.splitext(file_path)[1]=='.json':
    f = open(file_path)
    f_json = json.load(f)
    try:
      abstract = f_json['abstract'][0]['text']
    except:
      error_count+=1
    for i in f_json['body_text']:
      try:
        body_text= body_text+' '+i['text']
      except:
        error_count+=1
    body_text = body_text.strip()
    f.close()
    return body_text,abstract,error_count
  else:
    return body_text,abstract,error_count

## Storing article and related information in data-frame
df = pd.DataFrame({'file_name':[],'body':[],'abstract':[],'error_count':[]})
df['file_name'] = file_names
df['article_no'] = list(range(df.shape[0]))
for ind,info in tqdm(df.iterrows(),total=df.shape[0]):  df.loc[ind,'body'],df.loc[ind,'abstract'],df.loc[ind,'error_count'] = \
  file_content(file_path=info['file_name'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: ignored

In [ ]:
df.head(3)

In [ ]:
df.info()

Now, all articles stored in data-frame with columns
file_name: containing absolute file path (unique)
>1. **body:** Content of article<br>
>2. **abstract:** Abstract of article<br>
>3. **error_count:** Continuous value tells if any error came while parsing the file<br>
>4. **article_no:** Unique Identification number for each article, simply number starting from 0

### 3. Preprocessing functions
Below is code for some common preprocessing (lemmatization, stopword removal) steps used

In [ ]:
corpus_file = 'corpus.txt'
sent_dict_file = 'sent.joblib.compressed'
word_sent_no_dict_file = 'word_sent_no.joblib.compressed'
orig_word_sent_no_dict_file = 'orig_word_sent_no.joblib.compressed'
stopword_file = 'stopword.txt'

In [ ]:
## Lemmatization function
def get_wordnet_pos(word):
  """Map POS tag to first character lemmatize() accepts"""
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_tokenize(sent)])

def parallelize_dataframe(df, func, num_partitions, num_cores):
  df_split = np.array_split(df, num_partitions)
  pool = Pool(num_cores)
  df = pd.concat(pool.map(func, df_split))
  pool.close()
  pool.join()
  return df
  
def fn_lemmatize(data):
  for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
    data.loc[ind,'sentence_lemmatized'] = get_lemmatize(sent = info['sentence'])
  return data

In [ ]:
## removing stopwords
def words(text): 
  return re.findall(r'\w+', text.lower())
  
#stopwords = list(set(words(open(stopword_file).read())))

def remove_stopwords(sent):
  ## case conversion - lower case
  word_tokens = words(text=sent)
  #sent = sent.lower()
  #word_tokens = word_tokenize(sent)
  ## removing stopwords
  filtered_sentence = " ".join([w for w in word_tokens if not w in stopwords])
  ## removing digits
  filtered_sentence = re.sub(r'\d+','',filtered_sentence)
  ## removing multiple space
  filtered_sentence = words(text = filtered_sentence)
  return " ".join(filtered_sentence)

def fn_stopword(data):
  for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
    sent = info['sentence_lemmatized']
    data.loc[ind,'sentence_lemma_stop'] = remove_stopwords(sent)
  return data

def fn_stopword_orig(data):
  for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
    sent = info['sentence']
    data.loc[ind,'sentence_stop'] = remove_stopwords(sent)
  return data

### 4. Convert data in to useful format
>**Sentence dictionary**: Split all body and abstract from above dataframe into sentences using ```nltk.sent_tokenize``` and store in dictionary where sentence number is key and value is sentence. It will be used to retrieve sentences given sentence number.

In [ ]:
## creating sentence dictionary
df['article'] = df['body']+' '+ df['abstract']
df['article'].fillna('',inplace=True)
article_no_sent_dict = dict()

for ind,info in tqdm(df.iterrows(),total=df.shape[0]):
  article_no_sent_dict[info['article_no']] = sent_tokenize(str(info['article']))
  
article_no_list = list();sent_list = list()
df_sent = pd.DataFrame({'article_id':[],'sentence':[]})
for i in tqdm(article_no_sent_dict,total=len(article_no_sent_dict)):
  article_no_list.extend([i]*len(article_no_sent_dict[i]))
  sent_list.extend(article_no_sent_dict[i])
df_sent['article_id'] = article_no_list ; df_sent['sentence'] = sent_list
df_sent['sent_no'] = list(range(df_sent.shape[0]))
## sentence level dictionary
sent_dict = dict()
for ind,info in tqdm(df_sent.iterrows(),total=df_sent.shape[0]):
  sent_dict[info['sent_no']] = info['sentence']
sent_dict[-1] = 'NULL'
sent_dict_file = 'sent.joblib.compressed'
joblib.dump(sent_dict,sent_dict_file, compress=True)

In [ ]:
df_sent.head(5)

In [ ]:
df_sent.info()

Lemmatized Word — Sentence No. Inverse dictionary: <br>Create dictionary with key as lemmatized word from sentence and value as list of sentence number where lemmatized word is present. It will be used to retreive list of sentence number given lemmatized word.

In [ ]:
## lemmatization over sentence

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

df1 = deepcopy(df_sent)
df1 = parallelize_dataframe(df=df1, func=fn_lemmatize, num_partitions=27, num_cores=27)
## removing stopword from lemmatized sentence
df1 = parallelize_dataframe(df=df1, func=fn_stopword, num_partitions=30, num_cores=35)
## saving inverse dictionary on lemmatized sentence i.e. word and sentence no
word_sent_no_dict = defaultdict(list)
for ind,info in tqdm(df1.iterrows(),total=df1.shape[0]):
  sent_words = words(info['sentence_lemma_stop'])
  for w in sent_words:
    word_sent_no_dict[w].append(info['sent_no'])
joblib.dump(word_sent_no_dict,word_sent_no_dict_file, compress=True)

### **Original Word **— Sentence No.<br>
 Inverse dictionary: Create dictionary with key as original word from sentence and value as list of sentence number where lemmatized word is present. It will be used to retreive list of sentence number given original word.

In [ ]:
## saving inverse dictionary on original sentence i.e. word and sentence no
df1 = deepcopy(df_sent)
df1 = parallelize_dataframe(df=df1, func=fn_stopword_orig, num_partitions=35, num_cores=35)
orig_word_sent_no_dict = defaultdict(list)
for ind,info in tqdm(df1.iterrows(),total=df1.shape[0]):
  sent_words = words(info['sentence_stop'])
  for w in sent_words:
    orig_word_sent_no_dict[w].append(info['sent_no'])
joblib.dump(orig_word_sent_no_dict,orig_word_sent_no_dict_file, compress=True)

### **Corpus.txt:** It contains all sentences in single .txt file, will be used as reference for query spelling correction purpose.

In [ ]:
## Corpus - for spelling correction model
outF = open(corpus_file, "w")
for line in tqdm(df_sent['sentence'],total=df_sent.shape[0]):
 # write line to output file
 outF.write(line)
 outF.write("\n")
outF.close()

In [ ]:
import os
os.listdir('.')

### 5. Query Processing System

### 6. Filter relevant Sentences based on Query

### 7. Use BERT Model for QA System

In [ ]:
!pip install transformers
from transformers import pipeline


nlp = pipeline('question-answering',model = 'bert-large-cased-whole-word-masking-finetuned-squad')

query_sample = "How to prevent Corona ?"
relevant_sentence = "When asked why they were wearing masks, several students answered that they were 'preventing corona'."


nlp(question = query_sample, context = relevant_sentence)

In [ ]:
#query_sample = input("Enter question:") 
#print(query_sample) 
#nlp(question = query_sample)